In [26]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

# camerons api key
API_KEY = 'KrlCoAsmMTGJftfD6uV-s7kSN0o-SG2anMcTrwP6G4szLC3AsKX-hbiVLqiBfMj371sNemBCzrN8zCHae5SgIp1jqoiExhvExitIroX4_mBzQgWVtxcXFdFQIkkNYnYx'
yelp_api = YelpAPI(API_KEY)

In [27]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [28]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [29]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning
base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/' and removes '[]'
def cleaned(lst):
    for neighborhood in lst:
        neighborhood = str(neighborhood)
        if '/' in neighborhood or ' / ' in neighborhood:
            lst.remove(neighborhood)
            aliases = neighborhood.split('/')
            for item in aliases:
                lst.append(item.strip())
        if '[' in neighborhood and neighborhood in lst:
            lst.remove(neighborhood)
            split = neighborhood.split('[')
            lst.append(split[0].strip())
    return lst

In [30]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# scraping chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = cleaned(chicago_neighborhoods)

# cleaning chicago neighborhoods

# # Replace 'Lakeview (East)' with Lake View East
# # Replace Legends South (Robert Taylor Homes) with Legends South, Robert Taylor Homes
# # Replace 'Lakeview (West)' with Lake View West
# neighborhoods['Chicago, Illinois'].replace('Lakeview (East)', 'Lakeview East')
# # neighborhoods['Chicago, Illinois'] = neighborhoods['Chicago, Illinois'].replace('Lakeview (West)', 'Lakeview West')
# # neighborhoods['Chicago, Illinois'] = neighborhoods['Chicago, Illinois'].replace('Legends South (Robert Taylor Homes)', 'Legends South')
# # neighborhoods['Chicago, Illinois']

# scraping indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[1]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = cleaned(indianapolis_neighborhoods)

# scraping st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = cleaned(stl_neighborhoods)

# scraping detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

In [31]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# scraping columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = cleaned(columbus_neighborhoods)

# scraping milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = cleaned(mlwk_neighborhoods)

# scraping kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = cleaned(kansas_neighborhoods)

# scraping minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = cleaned(minneapolis_neighborhoods)

# scraping saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = cleaned(stpaul_neighborhoods)

In [37]:
# list of chains/fast food restaurants to remove
chains = pd.read_csv('cleaning/detroit_chains.csv')
dtw_chains = pd.read_csv('cleaning/detroit_local_chains.csv')
chains_names = list(chains['name'])
dtw_chains_names = list(dtw_chains['name'])
chains_names

['Wahlburgers',
 'IHOP',
 'Buffalo Wild Wings',
 "Salsarita's Fresh Mexican Grill",
 'Potbelly Sandwich Shop',
 'HopCat',
 'Texas de Brazil',
 'Shake Shack',
 'Blimpie',
 'Wingstop',
 'White Castle',
 'Boston Market',
 "McDonald's",
 'KFC',
 'Little Caesars Pizza',
 "McDonald's",
 "Jet's Pizza",
 'Burger King',
 "Domino's Pizza",
 'Little Caesars Pizza',
 'KFC',
 "McDonald's",
 'Popeyes Louisiana Kitchen',
 "Church's Chicken",
 'Burger King',
 "Wendy's",
 'Little Caesars',
 'White Castle',
 "Jet's Pizza",
 "McDonald's",
 "Wendy's",
 'Little Caesars',
 "Happy's Pizza",
 "Church's Chicken",
 'Little Caesars Pizza',
 "Wendy's",
 "Hungry Howie's Pizza",
 'Subway',
 'Subway',
 "Tubby's",
 'White Castle',
 "Wendy's",
 'Popeyes Louisiana Kitchen',
 "McDonald's",
 'Subway',
 'KFC',
 'Little Caesars Pizza',
 'Little Caesars Pizza',
 'Subway',
 'Burger King',
 "Church's Chicken",
 'Five Guys',
 "Domino's Pizza",
 "Church's Chicken",
 'Burger King',
 "Wendy's",
 'Subway',
 'White Castle',
 'KFC',

In [38]:
# STEP 2.1: get black owned restaurant id list for Detroit - used to create a column in csv file with Yelp API

# black_owned_dtw = []

# url = "https://api.yelp.com/v3/businesses/search"
# headers = {'Authorization': f'Bearer {API_KEY}', 'user-agent': 'Mozilla/5.0'}

# for n in neighborhoods['Detroit, Michigan']:
#     params = {
#         "category": "restaurants",
#         "location": f'{n}, Detroit',
#         "attributes": "is_black_owned"
#     }
#     try:
#         response = requests.get(url, params=params, headers=headers).json()['businesses']
#         for business in response:
#             if business['id'] not in black_owned_dtw:
#                 black_owned_dtw.append(business['id'])
#             else:
#                 continue
#     except KeyError:
#         continue
        
# len(black_owned_dtw)

In [39]:
# STEP 2.1: get women restaurant id list for Detroit - used to create a column in csv file with Yelp API

# women_owned_dtw = []

# url = "https://api.yelp.com/v3/businesses/search"
# headers = {'Authorization': f'Bearer {API_KEY}','user-agent': 'Mozilla/5.0'}

# for n in neighborhoods['Detroit, Michigan']:
#     params = {
#         "category": "restaurants",
#         "location": f'{n}, Detroit',
#         "attributes": "is_women_owned"
#     }
#     try:
#         response = requests.get(url, params=params, headers=headers).json()['businesses']
#         for business in response:
#             if business['id'] not in women_owned_dtw:
#                 women_owned_dtw.append(business['id'])
#             else:
#                 continue
#     except KeyError:
#         continue
        
# len(women_owned_dtw)

In [40]:
# STEP 2.1: get women restaurant id list for Detroit - used to create a column in csv file with Yelp API

# black_owned_dtw = []
# offset = 0
# count = 0

# for n in neighborhoods['Detroit, Michigan']:
#     try:
#         while count <= 450:
#             result = yelp_api.search_query(categories='restaurants', location=n + ', Detroit', limit=50, offset=offset, attributes='is_black_owned')['businesses']
#             for business in response:
#                 if business['id'] not in black_owned_dtw:
#                     black_owned_dtw.append(business['id'])
#                     print(business['name'])
#                     count += 1
#                     offset += 50
#                 else:
#                     continue
#     except KeyError:
#         continue
        
# len(black_owned_dtw)

In [41]:
# STEP 2.3: use yelp API to get restaurant data for each neighborhood and write results into csv file

for city in cities:
    previous_restaurant_ids = []
    city_name = city.lower().split(',')[0]
    city_file_name = '_'.join(city_name.split()).replace('.', '')
    with open(f'{city_file_name}_restaurants.csv', "w", newline="") as fileout:
        header = fileout.write("id,name,review_count,rating,latitude,longitude,price,zip_code\n") # add headers
        writer = csv.writer(fileout)
        count = 0
        for n in neighborhoods[city]:
            offset = 0
            try:
                # obtaining up to 1000 restaurants total restaurants
                while offset <= 1000:
                    result = yelp_api.search_query(categories='restaurants', location=n + f', {city_name}', limit=50, offset=offset)['businesses']
                    # parsing through json
                    current_idx = 0
                    for business in result:
                        current_idx += 1
                        restaurant_id = business['id']
                        name = business['name']
                        review_count = business['review_count']
                        rating = business['rating']
                        zip_code = business['location']['zip_code']
                        try:
                            latitude = str(round(business['coordinates']['latitude'], 5))
                            longitude = str(round(business['coordinates']['longitude'], 5))
                        except TypeError:
                            latitude = None
                            longitude = None
                        try:
                            price = business['price']
                        except KeyError:
                            price = None
                        if restaurant_id in previous_restaurant_ids or name in chains_names:
                            print('Skipping duplicates,fast food,chains...')
                            offset += current_idx
                            continue
                        else:
                            row = restaurant_id,name,review_count,rating,latitude,longitude,price,zip_code
                            writer.writerow(row)
                            count += 1
                            print(name + ' - '+ str(count))
                            previous_restaurant_ids.append(restaurant_id)                  
                    offset += 50    
            # way to handle errors if neighborhood does not give search results with Yelp API
            except YelpAPI.YelpAPIError as e:
                print('Error')
                continue
        print('Done')

Lawrence Fish Market - 1
Smoque BBQ - 2
Noon O Kabab - 3
Kabobi Grill - 4
Bryn Mawr Breakfast Club - 5
Great Sea Restaurant - 6
Ruby's Fast Food - 7
Torteria San Lenchito - 8
Kuma's Corner - Belmont - 9
Tre Kronor - 10
Nhu Lan Saigon Subs - 11
Hot Dog Station - 12
Dulce De Leche Cafe - 13
LD Pho - 14
Ssyal Ginseng House - 15
Salam Restaurant - 16
Mother Cluckers Kitchen - Chicago - 17
Wabi Sabi - 18
Gather - 19
Monti's - 20
Shokran Moroccan Restaurant - 21
Chicago Kalbi - 22
SUBO Filipino Kitchen - 23
Angelo's Wine Bar - 24
Hala In Restaurant - 25
Kale My Name - 26
Goosefoot - 27
The Bad Apple - 28
Rojo Gusano - 29
Shibam Restaurant - 30
Crepe House Cafe - 31
Brasa Roja - 32
Honey Butter Fried Chicken - 33
First Slice - 34
Old Irving Brewing - 35
Peking Mandarin - 36
Longman & Eagle - 37
Community Tavern - 38
Sushi Joon - 39
Jimmy's Pizza Café - 40
Little Bad Wolf - 41
La Nonna - 42
Ban Po Jung - 43
Spacca Napoli - 44
Lula Cafe - 45
Patino's Grill - 46
Luella's Southern Kitchen - 47
No

Mas Alla Del Sol - 401
Hugo's Frog Bar & Fish House - 402
Sushi + Rotary Sushi Bar - Boystown - 403
Friendship Chinese Restaurant - 404
Park & Field - 405
Brownstone Tavern & Grill - 406
Cozy Noodles & Rice - 407
Steak 48 - 408
Geja's Cafe - 409
Topo Gigio Ristorante - 410
Indie Cafe - 411
Four Moon Tavern - 412
Vini's Pizza - 413
Siam Noodle and Rice - 414
Chim Thai Cuisine - 415
Jibaritos y Más - 416
Julius Meinl Chicago - 417
Rainbow Cuisine - 418
Bayan Ko - 419
Mayfair Restaurant - 420
Fin Sushi Bar - 421
North Branch Fried Chicken - 422
Taza Bakery - 423
Yellow Rose Cafe - 424
Outdoor Grill - 425
Simòn Tacos - 426
RYUU Asian BBQ & Sushi - 427
Cairo Kebab - 428
Psistaria Greek Taverna - 429
El Asadero Colombiano - 430
Dimo's Pizza - 431
Señor Pan Cafe - 432
Uptown Pho - 433
O'Donovan's - 434
Amitabul - 435
Pete's Pizza - 436
Cafe Tola - 437
Le Sud - 438
Grand Lux Cafe - Chicago - 439
Taqueria Moran - 440
Paulie Gee's Logan Square - 441
Furious Spoon - Logan Square - 442
Easy Street

Campagnola - 749
Buena Vista - 750
Finley Dunnes Tavern - 751
Hangry's - 752
Tiffin - 753
Dollop Coffee - 754
Ann Sather Restaurant & Catering - 755
SABA Italian Bar + Kitchen - 756
Martino's Italian Cuisine & Pizzeria - 757
Hon Kee Bbq & Seafood Restaurant - 758
Uncommon Ground - 759
Carmine's - 760
First Sip Cafe - 761
Alegrias Seafood - 762
Funkenhausen - 763
The Harding Tavern - 764
Krung Thep Thai Cuisine - 765
La Crosta Woodfire Pizzeria Italiana - 766
The Wiener's Circle - 767
Savanna Restaurant - 768
Jin Thai - 769
Sushi Mon - 770
Skipping duplicates, fast food/chains...
Golden Nugget Pancake House - 771
Massa Cafe Italiano - 772
Zam Zam Middle Eastern Grill - 773
Taqueria Traspasada - 774
Shinsen Evanston - 775
Brazilian Bowl - 776
Cafe Marbella - 777
Mee Mah Restaurant - 778
Tzuco - 779
Eduardo's Enoteca - 780
The Reservoir - 781
Centennial Crafted Beer & Eatery - 782
The Piggery - 783
Bunny Hutch - 784
Antico - 785
Headquarters Beercade River North - 786
Pho's Spicier Thai C

Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplica

Error
MingHin Cuisine - 1220
Franco's Ristorante - 1221
Ricobene's - 1222
Happy Lamb Hot Pot - 1223
Nana - 1224
Pleasant House Pub - 1225
Qing Xiang Yuan Dumplings - 1226
MCCB Chicago 时尚食谱 - 1227
Chi Cafe - 1228
Gio's Cafe and Deli - 1229
Han 202 - 1230
Go 4 Food - 1231
Joy Yee Noodle - 1232
Hello Jasmine - 1233
Min's Noodle House 渔家重庆小面 - 1234
Chiu Quon Bakery & Dim Sum - 1235
Stix n Brix Wood Fired Pizza - 1236
Strings Ramen Shop - 1237
Maxwell Street Depot - 1238
Dolo Restaurant and Bar - 1239
Carbon - 1240
Bridgeport Coffee - 1241
Legend Tasty House - 1242
Connie's Pizza - 1243
Zaytune Mediterranean Grill - 1244
Pancho Pistolas - 1245
Triple Crown Restaurant - 1246
Lao Sze Chuan - 1247
Taipei Cafe - 1248
Qiao Lin Hotpot - 1249
Potsticker House - 1250
Yummy Yummy Noodles - 1251
Lou Malnati's Pizzeria - 1252
S.K.Y. - 1253
Sushi + Rotary Sushi Bar - Chinatown - 1254
Del Toro - 1255
Yolk - South Loop - 1256
Hing Kee Restaurant - 1257
Fabulous Freddies Italian Eatery - 1258
Daebak Korea

...

Ifoods Deli - 298
Rio Brazilian Steak House - 299
Mama Ann's Soul Food - 300
Nepali Kitchen - 301
Fortune House - 302
Michael's Pizza - 303
iPierogi - 304
City View Grille - 305
Skipping duplicates, fast food/chains...
R A MacSammy's Truck - 306
Skipping duplicates, fast food/chains...
ROK Eatery - 307
Skipping duplicates, fast food/chains...
Pimento Jamaican Kitchen - 308
Twin Cities Burger Battle - 309
Leeann Chin - 310
Francis - 311
Skipping duplicates, fast food/chains...
Redd Peppers Jamacain Food - 312
Panera Bread - 313
Rico's Ice Cream Parlor - 314
Dock & Paddle - 315
Papa Murphy's - 316
JW's Bierstube - 317
Panini Pinups - 318
Panda Express - 319
Rocco's Pizza - Oakdale - 320
Santa Fe - 321
Knowlan's Fresh Foods - 322
Headwaters Cafe - 323
East River Eats - 324
Twin Cities Crepes & Waffles - 325
Skipping duplicates, fast food/chains...
Green & Grill Global Taste - 326
Bella Ciao Pasta and Bar Ristorante - 327
Jimmy John's - 328
Happy Street Foods - 329
Red's Savoy Pizza - 330


Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Casper's and Runyon's Nook - 349
Skipping duplicates, fas

Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplica

Skipping duplicates, fast food/chains...
Bruegger's Bagels - 423
Vina Vietnamese - 424
Natedogs - 425
Moose Bar & Grill - 426
Pizza Biga - 427
Momo Dosa Street Food - 428
Katar River Restaurant and Bakery - 429
Fuzzy's Taco Shop Taqueria - 430
Tavola Italian Kitchen & Bar - 431
Roti - 432
Adam's Soul To Go - 433
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Coconut Whisk Cafe & Boba Shop - 434
New China Express - 435
Casa Maria - 436
Andrea Pizza - 437
Bakers Square - 438
Pinoy Fusion - 439
The Anchor Fish & Chips Food Truck - 440
Skipping duplicates, fast food/chains...
Joe & The Juice - 441
Skipping duplicates, fast food/chains...
Denny's - 442
Pracna on Main - 443
A&J Fish & Chicken - 444
Skipping duplicates, fast food/chains...
The Corner Bar - 445
Bunker's Music Bar & Grill - 446
Sawatdee Thai Restaurant - Minneapolis - 447
Tipsy Steer - 448
Skipping duplicates, fast food/chains...
Davanni's Pizza & Hot Hoagies - 449
Skipping duplicates, fast fo

Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplicates, fast food/chains...
Skipping duplica

In [ ]:
# updates: made csv files for all restaurants - working on removing fast food / chain restaurnts

# questions / comments:
    # rounding?
    # having trouble with attributes - return same values
    # cam - is parser applicable to all midwest cities' chains? is one too restrictive?
    # skip duplicates by alias/nam vs id?